---
# Unit08: The Network and Array Methods

This notebook has the activities of the Course **ProSeisSN**. It deals with time series processing using a passive seismic dataset using [ObsPy](https://docs.obspy.org/).

#### Dependencies: Obspy, Numpy, Matplotlib
#### Reset the Jupyter/IPython notebook in order to run it again, press:
***Kernel*** -> ***Restart & Clear Output***
#### The code $\Downarrow$ BELOW $\Downarrow$ runs a notebook with other dependencies

In [ ]:
#------ Import Libraries
import sys
import os
    
#------ Work with the directory structure to include auxiliary codes
print('\n Local directory ==> ', os.getcwd())
print('  - Contents: ', os.listdir(), '\n')

path = os.path.abspath(os.path.join('..'))
if path not in sys.path:
    sys.path.append(path+"/CodePy")

%run ../CodePy/ImpMod.ipynb

#------ Alter default matplotlib rcParams
from matplotlib import rcParams
import matplotlib.dates as dates
# Change the defaults of the runtime configuration settings in the global variable matplotlib.rcParams
plt.rcParams['figure.figsize'] = 9, 5
#plt.rcParams['lines.linewidth'] = 0.5
plt.rcParams["figure.subplot.hspace"] = (.9)

#------ Magic commands
%matplotlib inline
%matplotlib widget
#%pylab notebook

---
## Read **file 3804** data from TTB22
- **The Receiver Array**
The array has 24 GS-20DX vertical geophones hooked to a $L=69$m cable, using takeouts spaced $\delta l=6$m from each other. The GS-20DX geophones have a natural frequency of $f_n=10\textrm{Hz}$, and a spurious frequency $f_{sp}>250$Hz. The array has a irregular circular shape, deployed in the Southern tip of the island, with its center at $\left(1^{\circ}12^{\prime}6.93^{\prime\prime}\textrm{S},48^{\circ}30^{\prime}23.39^{\prime\prime}\textrm{S}\right)$.
- **The data**
Each of the 12 traces of **file 3804** is $\Delta T=60$s long, with a sampling frequency of $f_{s}=250$Hz. This file was recorded on 2022-04-02, begining at 13h 56min 41s. That was during the maximum gradient of the local ebb tide.

<img src="./ttb_tide.png" width="600">

In [ ]:
"""
====================== READ THE SEISMIC DATA LOCALLY ======================
"""
#
#------ Read the seismic data
filename = '3804'
print(f">> Read with data file {filename}")
filename = '../Data/'+filename+'.dat'   
#------- Read the data file as a SEG2 object
st = read(filename)
#
#------- Print stream information
dummy = float(st[-1].stats.seg2.RECEIVER_LOCATION)
print(f"1) Gather acquired on {st[0].stats.starttime}, has {len(st)} geophones along {dummy}m.")
dummy = (UTCDateTime(st[0].stats.endtime) - UTCDateTime(st[0].stats.starttime))
print(f"2) Each {dummy}s-long trace has {int(st[0].stats.npts)} data points.")
print(f"3) The sampling frequency is {st[0].stats.sampling_rate}Hz")
#
"""
====================== READ PHONES LOCATIONS ======================
"""
#------ Read the phones cartesian locations
#--- Reads the CSV file with (x, y)m locations
ttb_loc = u.RGloc('../Data/'+'ttb_loc.dat')
#------ Read the phones geographic locations
#--- Reads the CSV file with (lat,lon) in degress locations
ttb_gloc = u.RGloc('../Data/'+'ttb_gloc.dat')
#
#------ Plot gather in cartesian
p.pgather(ttb_loc[:,1], ttb_loc[:,2], ttb_loc[:,0], coord='cartesian')

---
## Display all the data as a seismogram
- Use the experience with one trace to process the whole stream together.
- A distance dependent plot shows the different move-out of seismic arrivals and gives an idea of the  backazimuth and slowness that could be expected.

In [ ]:
#
#------ Create a new stream from the SEG2 stream. Gather baricenter = bcenter
gather, bcenter = u.creastrm(st, ttb_gloc)
#------- Filter the stream
#--- Filter parameters
MTparam = [1, 1, 'bp',  5., 50., 1, 0]
# └─────> [dtr, line, ftype, Fmin, Fmax, taper, gain]
gather = u.otrstr(gather, MTparam)
#
#------ Plot
gather.plot(type='section',
            scale=1.3, alpha=.7,
            orientation='horizontal')

#### $\Downarrow$ Zoom in the above seismogram $\Downarrow$

In [ ]:
#
#------ Zoom in the seismogram
ent = input(f' Enter t0 and t1 to zoom: ')
ent = ent.rstrip().split(' ')
f0 = float(ent[0])
f1 = float(ent[1])
#
dt = gather[0].stats.starttime
gather.plot(type='section',
            scale=1.3, alpha=.7,
            starttime=dt+f0, endtime=dt+f1,
            orientation='horizontal')

---
## Matched-filter detection
- Create a catalog of **template waveforms** from file 3804 to generate characteristic functions that could be used for detection.
- Once the catalog is created it is necessary to provide two paramters:
1) The correlation threshold within the normalised correlation range [-1, 1]
2)  The triger off time in s. This is the minimum distance between events.
#### The code $\Downarrow$ BELOW $\Downarrow$ resets the event catalog to an empty list: it should be run at least once.

In [ ]:
#------ Reset event catalog
try:
    del cat
except:
    cat   = [] #-> wavefrom catalog
    tname = [] #-> template names
print(f">> Event catalog reset")

#### The code $\Downarrow$ BELOW $\Downarrow$ can be run many times to build the catalog

In [ ]:
"""
Build a catalog of templates. 
"""
#------ Construct catalog
#------ Pick up a phone
dummy = np.random.randint(1, len(st)+1)
ent = input(f' Enter a phone [dflt=random] ')
ent = int( ent.rstrip().split(' ')[0] ) if ent else dummy
#------- The trace = Phone -1.
trZ = gather[ent - 1].copy()
#
#------ Plot Spectrogram
time = trZ.times(type="relative")
p.Pspect(time, trZ)
#
#-------- Construct catalog of template waveforms with names
cat, tname = u.cat_wfrm(trZ, cat, tname, ent)
#
print(f">> Template catalog has {len(cat)} templates: {tname}")

---
## Detect events with the Matched-filter detection algorithms
Work with a single trace. It is necessary to provide two paramters:
- The correlation threshold within the normalised correlation range [-1, 1]
- The triger off time in s. This is the minimum distance between events.

In [ ]:
#------ Import Packages
from obspy.signal.cross_correlation import correlation_detector
import pprint
print(f">> Trigger packages imported.")
#
#------ Enter height and distance (s). Hint: .6 1
ent = input(f'<< Enter correlation threshold and triger off time(s): ')
ent = ent.rstrip().split(' ')
corthr   = float(ent[0])
trgoff   = float(ent[1])
#
stream = Stream()
for i in range(0, len(gather), 6):
    stream  = Stream([gather[i]])
#
#------ Detections. Free memory on each loop.
    detect = []
    for template in cat:
        tmplstr = Stream([template])
        dets, sims = correlation_detector(
                       stream=stream, templates=[tmplstr],
                       heights=corthr, distance=trgoff, plot=stream,
                       template_names=tname )
        detect.extend(dets)
#
    print("Trace {0} has {1} detections from {2} templates".format(i+1, len(detect), len(cat)))
    print(detect)

### Computing DOA with sliding-window array processing

The next code block applies the least-squares method for estimating the DOA in a sliding-window. You can specify the start and end time to process, the length of the time window, and the overlap. The output is a list of times, phase velocities, and backazimuths. The subsequent plot shows the results with the beamformed waveform for comparison.

In [ ]:
#
#------ Get the array coordinates
X, stn = u.acoord(gather, bcenter)
#
#------ Enter the sliding-window parameters
print(f'>> Provide the parameters for estimating the DOA with a sliding-window:')
print(f'   1) twin         -> length of the time window;')
print(f'   2) twstep       -> the window step')
print(f'   3) tstart, tend -> the relative start and end times to process;')
#
ent = input(f'<< Enter twin, twstep, tstart, and tend: ') or '1.2 0.2 30 40'
twin, twstep, tstart, tend = np.array(ent.rstrip().split(' '), dtype=float)
#
#------------ 
slid_fk = u.sldtw_fk(gather, tstart, tend, MTparam, win_len=twin, win_frac=twstep)
#------------ Convert times to seconds after Stream start time:
#T = (slid_fk[:,0] - date2num(st[0].stats.starttime.datetime))*86400


#T = (slid_fk[:,0] - date2num(gather[0].stats.starttime.datetime))*86400
T = slid_fk[:,0] - gather[0].times("matplotlib")[0]


#------------ Convert backazimuths to degrees from North:
B = slid_fk[:,3] % 360.
#------------ Convert slowness to phase velocity:
V = 1/slid_fk[:,4]
#------------ Semblance:
S = slid_fk[:,1]
#------------ Plot. Mwax velocity is vmax
vmax =  1.5

ent = input(f'<< Enter a phone: ')
ent = ent.rstrip().split(' ')
p.psw(gather, T, B, V, C=S, v_max=vmax, element=ent[0], twin_plot=[tstart,tend])

#
#------ Get a list of times, phase velocities, and backazimuths
#T, V, B = u.sldtw(gather, X, tstart, tend, twin, overlap)
#
#------ Plot
#st = u.add_beam(gather, bcenter, 'bcenter')
#p.psw(gather, T, B, V, v_max=1., twin_plot=[tstart,tend])


In [ ]:
ent = int(ent) - 1
flims = [MTparam[3], MTparam[4]]
"""
====================== BEAMFORMING ======================
"""
stream = Stream()
dummy  = Stream([gather[ent]])
print(len(dummy))
# ---------- Use FK Analysis
out, stime, etime = u.BeamFK(dummy, flims, ttb_gloc)
#-- Make output human readable
t, rel_power, abs_power, baz, slow = out.T
#------------- print
sys.stdout.write('\n')
print(f'>> t       rel_power abs_power   baz(deg) slow(s/km)')    
for i in range(len(t)):
    print(f'   {round(t[i],4)}, {round(rel_power[i],4)}, {round(abs_power[i],4)}, {round(baz[i],4)}, {round(slow[i],4)}')    
#------------ Plot
p.pltbaz(out, stime , etime)



---
